In [16]:
import requests
from bs4 import BeautifulSoup
from collections import defaultdict

# Function to scrape Redfin rental listings for a given zip code
def scrape_redfin_rentals(zip_code):
    base_url = f"https://www.redfin.com/zipcode/{zip_code}/apartments-for-rent"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(base_url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.find_all('div', class_='HomeCardContainer')

# Function to extract the number of beds and baths, and rent price from a rental listing
def extract_rental_info(listing):
    print(listing)
    beds = listing.find('div', class_='HomeStatsV2').find_all('div')[0].text.strip()
    baths = listing.find('div', class_='HomeStatsV2').find_all('div')[1].text.strip()
    #print(listing.find('div', class_='homecardV2Price').text)
    #price = listing.find('div', class_='homecardV2Price').text.strip().replace('$', '').replace(',', '')
    price = listing.find_all('span', {'class' : 'homecardV2Price'})[0].text.strip().replace('$', '').replace(',', '').replace('/mo', '')
    return beds, baths, int(price)

# Function to calculate the average rent price for each number of beds and baths
def calculate_average_rent(zip_code):
    rentals = scrape_redfin_rentals(zip_code)
    bed_bath_prices = defaultdict(list)

    for rental in rentals:
        beds, baths, price = extract_rental_info(rental)
        bed_bath_prices[(beds, baths)].append(price)

    average_rent = {}
    for key, value in bed_bath_prices.items():
        average_rent[key] = sum(value) / len(value)

    return average_rent

# Specify the zip code you want to scrape
zip_code = '92082'

# Calculate the average rent price for each number of beds and baths
average_rent_prices = calculate_average_rent(zip_code)

# Print the results
for key, value in average_rent_prices.items():
    beds, baths = keys
    print(f"Average rent for {beds} beds, {baths} baths: ${value:.2f}")


<div class="HomeCardContainer selectedHomeCard defaultSplitMapListView isRentals" data-rf-test-id="react-data-photo-row-0" id="MapHomeCard_0"><div class="MapHomeCardReact HomeCard" data-rf-paint-id="map" data-rf-test-name="mapHomeCard"><div class="v2 interactive" tabindex="0"><div class="homecardv2" data-rf-test-name="basicNode-homeCard"><div class="cover-all"><div class="PhotoSlider photoContainer"><div class="scrollable" style="transform:translateX(0px)"><div class="slider-item invisible" data-rf-test-id="slider-item--1" style="transform:translateX(0px)"><div class="remarks-container" role="button"><div class="lazyload-placeholder" style="height:100%"></div><div class="remarks withTitle"><p>92082 House for Rent:</p><p>Short term rental rates start at $599 per night with a 4 day min. Private estate with breathtaking view's. Includes landscape service &amp; all the organic orange's you can eat. Welcome to p</p></div></div></div><a class="slider-item" data-rf-test-id="slider-item-0" dat

AttributeError: 'NoneType' object has no attribute 'find_all'

In [10]:
#test

base_url = f"https://www.redfin.com/zipcode/{zip_code}/apartments-for-rent"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
response = requests.get(base_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
rentals = soup.find_all('div', class_='HomeCardContainer')

for rental in rentals:
    print(rental)
    beds = rental.find('div', class_='HomeStatsV2').find_all('div')[0].text.strip()
    baths = rental.find('div', class_='HomeStatsV2').find_all('div')[1].text.strip()
    print(rental.find('div', class_='homecardV2Price'))
    price = rental.find('div', class_='homecardV2Price').text.strip().replace('$', '').replace(',', '')


<div class="HomeCardContainer selectedHomeCard defaultSplitMapListView isRentals" data-rf-test-id="react-data-photo-row-0" id="MapHomeCard_0"><div class="MapHomeCardReact HomeCard" data-rf-paint-id="map" data-rf-test-name="mapHomeCard"><div class="v2 interactive" tabindex="0"><div class="homecardv2" data-rf-test-name="basicNode-homeCard"><div class="cover-all"><div class="PhotoSlider photoContainer"><div class="scrollable" style="transform:translateX(0px)"><div class="slider-item invisible" data-rf-test-id="slider-item--1" style="transform:translateX(0px)"><div class="remarks-container" role="button"><div class="lazyload-placeholder" style="height:100%"></div><div class="remarks withTitle"><p>92082 House for Rent:</p><p>Short term rental rates start at $599 per night with a 4 day min. Private estate with breathtaking view's. Includes landscape service &amp; all the organic orange's you can eat. Welcome to p</p></div></div></div><a class="slider-item" data-rf-test-id="slider-item-0" dat

AttributeError: 'NoneType' object has no attribute 'text'

In [15]:
print(rental.find_all('span', {'class' : 'homecardV2Price'})[0].text.strip().replace('$', '').replace(',', '').replace('/mo', '')) #find('div', span_='homecardV2Price'))

9600


In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

# Function to scrape Redfin rental listings for a given zip code
def scrape_redfin_rentals(zip_code):
    base_url = f"https://www.redfin.com/zipcode/{zip_code}/apartments-for-rent/filter/property-type=house"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(base_url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.find_all('div', class_='HomeCardContainer')

# Function to extract rental information from a listing
def extract_rental_info(listing):
    url = listing.find('a', class_='homecardV2')['href']
    address = listing.find('span', class_='street-address').text.strip()
    price = listing.find('span', class_='homecardV2Price').text.strip().replace('$', '').replace(',', '')
    beds = listing.find('span', class_='data-beds').text.strip()
    baths = listing.find('span', class_='data-baths').text.strip()
    sqft = listing.find('span', class_='data-sqft').text.strip().replace(',', '')
    current_date = datetime.date.today().strftime('%Y-%m-%d')
    return url, address, int(price), beds, baths, sqft, current_date

# Function to calculate average rent prices for each number of beds and baths
def calculate_average_rent(prices):
    average_rent = {}
    for key, value in prices.items():
        total_rent = sum(value)
        avg_rent = total_rent / len(value)
        average_rent[key] = avg_rent
    return average_rent

# List of all zip codes in the United States
#all_zip_codes = range(10001, 99999)  # Modify range as per your requirement
all_zip_codes = [92082]

# Dataframe to store the rental information
rental_data = pd.DataFrame(columns=['Zip Code', 'URL', 'Address', 'Price', 'Beds', 'Baths', 'Sqft', 'Date'])

# Iterate through each zip code
for zip_code in all_zip_codes:
    rentals = scrape_redfin_rentals(zip_code)
    if rentals:
        prices = {}
        for rental in rentals:
            url, address, price, beds, baths, sqft, date = extract_rental_info(rental)
            rental_data = rental_data.append({
                'Zip Code': zip_code,
                'URL': url,
                'Address': address,
                'Price': price,
                'Beds': beds,
                'Baths': baths,
                'Sqft': sqft,
                'Date': date
            }, ignore_index=True)

# Calculate average rent prices for each zip code
average_rent_prices = rental_data.groupby(['Zip Code', 'Beds', 'Baths'])['Price'].mean().reset_index()

# Export dataframes to CSV for each zip code
for zip_code, group in average_rent_prices.groupby('Zip Code'):
    filename = f"rental_data_{zip_code}.csv"
    group.to_csv(filename, index=False)
    print(f"Data for Zip Code {zip_code} exported to {filename}")


TypeError: 'NoneType' object is not subscriptable